***GENERATED CODE FOR 2510ccardclass PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import pandas as pd
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        hdfs = HDFileSystem(host=hdfs_server, port=hdfs_port)
        with hdfs.open(eval(config)['url']) as f:
            dfPd = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run 2510ccardclassHooks.ipynb
try:
	#sourcePreExecutionHook()

	creditcardtrainsample = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/creditcardTrainSample.csv', 'filename': 'creditcardTrainSample.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'results_url': 'http://mluat.numtra.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run 2510ccardclassHooks.ipynb
try:
	#transformationPreExecutionHook()

	ccardclassautofe = TransformationMain.run(creditcardtrainsample,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "Time", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "29", "mean": "131.69", "stddev": "6.09", "min": "123", "max": "140", "missing": "0"}, "updatedLabel": "Time"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V1", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "29", "mean": "-0.2", "stddev": "1.51", "min": "-4.636533664", "max": "1.356489323", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V1"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V2", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "29", "mean": "-0.14", "stddev": "1.44", "min": "-5.931717441", "max": "1.687508027", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V3", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "29", "mean": "0.94", "stddev": "1.0", "min": "-1.333558486", "max": "2.974779375", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V3"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V4", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "29", "mean": "0.44", "stddev": "1.19", "min": "-2.737271346", "max": "2.801477073", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V4"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V5", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "29", "mean": "0.33", "stddev": "1.73", "min": "-1.421802565", "max": "7.672543966", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V5"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V6", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "29", "mean": "0.45", "stddev": "1.43", "min": "-1.23560826", "max": "4.091915383", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V6"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V7", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "29", "mean": "-0.09", "stddev": "0.92", "min": "-3.391184496", "max": "1.158792971", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V7"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V8", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "29", "mean": "0.17", "stddev": "0.48", "min": "-1.170683067", "max": "1.236266814", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V8"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V9", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "29", "mean": "0.25", "stddev": "0.83", "min": "-0.748284316", "max": "2.408675091", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V9"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V10", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "29", "mean": "0.06", "stddev": "0.83", "min": "-1.461860881", "max": "3.351079324", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V10"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V11", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "29", "mean": "0.05", "stddev": "0.97", "min": "-2.061908155", "max": "1.737926861", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V11"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V12", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "29", "mean": "0.34", "stddev": "0.6", "min": "-0.769377782", "max": "2.152055116", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V12"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V13", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "29", "mean": "0.0", "stddev": "0.89", "min": "-1.65017861", "max": "1.422912781", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V13"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V14", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "29", "mean": "-0.2", "stddev": "0.67", "min": "-2.44967118", "max": "0.778734284", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V14"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V15", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "29", "mean": "0.4", "stddev": "0.59", "min": "-1.182670988", "max": "1.430822389", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V15"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V16", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "29", "mean": "-0.18", "stddev": "0.85", "min": "-2.372316461", "max": "1.514765448", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V16"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V17", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "29", "mean": "-0.1", "stddev": "0.7", "min": "-1.312472695", "max": "1.614255302", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V17"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V18", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "29", "mean": "-0.3", "stddev": "0.73", "min": "-2.013090598", "max": "0.91642141", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V18"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V19", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "29", "mean": "0.19", "stddev": "0.77", "min": "-1.465499836", "max": "1.633690689", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V19"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V20", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "29", "mean": "0.15", "stddev": "0.55", "min": "-1.786736343", "max": "1.522136629", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V20"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V21", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "29", "mean": "-0.01", "stddev": "0.28", "min": "-0.657896819", "max": "0.655844388", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V21"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V22", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "29", "mean": "0.13", "stddev": "0.62", "min": "-0.838520084", "max": "1.461535075", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V22"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V23", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "29", "mean": "0.02", "stddev": "0.37", "min": "-0.438461573", "max": "1.304693094", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V23"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V24", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "29", "mean": "0.06", "stddev": "0.67", "min": "-1.441208864", "max": "1.063844582", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V24"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V25", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "29", "mean": "0.15", "stddev": "0.37", "min": "-0.809943468", "max": "0.837224847", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V25"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V26", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "29", "mean": "0.03", "stddev": "0.52", "min": "-0.733621849", "max": "1.286200506", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V26"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V27", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "29", "mean": "-0.01", "stddev": "0.19", "min": "-0.481570297", "max": "0.36241057", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V27"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V28", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "29", "mean": "-0.04", "stddev": "0.27", "min": "-1.325164299", "max": "0.237647042", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V28"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Amount", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "29", "mean": "64.53", "stddev": "80.36", "min": "1.29", "max": "257.3", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Amount"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Class", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "29", "mean": "0.24", "stddev": "0.44", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Class"}]}))

	#transformationPostExecutionHook(ccardclassautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run 2510ccardclassHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(ccardclassautofe, ["Time", "V1", "V2", "V3", "V4", "V5", "V6", "V7", "V8", "V9", "V10", "V11", "V12", "V13", "V14", "V15", "V16", "V17", "V18", "V19", "V20", "V21", "V22", "V23", "V24", "V25", "V26", "V27", "V28", "Amount"], "Class")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

